In [3]:
!pip3 install pandas-gbq -U
!pip3 install pandas
!pip3 install pandasql
!pip3 install google-cloud-bigquery --upgrade

Requirement already up-to-date: pandas-gbq in /usr/local/envs/py3env/lib/python3.5/site-packages (0.9.0)
Requirement already up-to-date: google-cloud-bigquery in /usr/local/envs/py3env/lib/python3.5/site-packages (1.9.0)


In [50]:
import pandas as pd

df = pd.read_csv("./data/donneesouvertes-interventions-sim.csv") 
df.head(n=5)

,incident_nbr,creation_date_time,incident_type_desc,description_groupe,caserne,nom_ville,nom_arrond,division,latitude,longitude,nombre_unites
0,1,2015-01-01 00:03:22,Inondation,Sans incendie,26,Montréal,Plateau Mont-Royal,11,45.535698,-73.580575,1.0
1,2,2015-01-01 00:05:58,Ac.véh./1R/s.v./ext/29B/D,Premier répondant,25,Montréal,Ville-Marie,10,45.494499,-73.574247,2.0
2,3,2015-01-01 00:08:34,Appel de Cie de détection,Alarmes-incendies,64,Montréal,Lachine,7,45.442005,-73.665779,3.0
3,4,2015-01-01 00:11:28,Premier répondant,Premier répondant,38,Montréal,Rivière-des-Prairies / Pointe-aux-Trembles,6,45.641294,-73.489981,1.0
4,5,2015-01-01 00:14:06,Odeur suspecte - gaz,Sans incendie,34,Montréal,Côte-des-Neiges / Notre-Dame-de-Grâce,9,45.483883,-73.615028,2.0


In [51]:
df['creation_date_time'] = pd.to_datetime(df['creation_date_time'])

L = ['year', 'month', 'day', 'hour', 'dayofweek']
df = df.join(pd.concat((getattr(df['creation_date_time'].dt, i).rename(i) for i in L), axis=1))

df.head(n=5)

,incident_nbr,creation_date_time,incident_type_desc,description_groupe,caserne,nom_ville,nom_arrond,division,latitude,longitude,nombre_unites,year,month,day,hour,dayofweek
0,1,2015-01-01 00:03:22,Inondation,Sans incendie,26,Montréal,Plateau Mont-Royal,11,45.535698,-73.580575,1.0,2015,1,1,0,3
1,2,2015-01-01 00:05:58,Ac.véh./1R/s.v./ext/29B/D,Premier répondant,25,Montréal,Ville-Marie,10,45.494499,-73.574247,2.0,2015,1,1,0,3
2,3,2015-01-01 00:08:34,Appel de Cie de détection,Alarmes-incendies,64,Montréal,Lachine,7,45.442005,-73.665779,3.0,2015,1,1,0,3
3,4,2015-01-01 00:11:28,Premier répondant,Premier répondant,38,Montréal,Rivière-des-Prairies / Pointe-aux-Trembles,6,45.641294,-73.489981,1.0,2015,1,1,0,3
4,5,2015-01-01 00:14:06,Odeur suspecte - gaz,Sans incendie,34,Montréal,Côte-des-Neiges / Notre-Dame-de-Grâce,9,45.483883,-73.615028,2.0,2015,1,1,0,3


In [36]:
df.shape

(390201, 15)

In [59]:
countsDataFrame = df.groupby(['division', 'year', 'month', 'day', 'hour']).size().reset_index(name='counts')

In [60]:
countsDataFrame.head(n=5)

,division,year,month,day,hour,counts
0,0,2015,1,1,13,1
1,0,2015,1,2,14,1
2,0,2015,1,3,14,1
3,0,2015,1,8,7,2
4,0,2015,1,13,13,1


In [64]:
import datetime
countsDataFrame['dayofweek'] = df.apply(lambda row: datetime.date(row['year'], row['month'], row['day']).weekday(), axis=1)
countsDataFrame.head(n=5)

,division,year,month,day,hour,counts,dayofweek
0,0,2015,1,1,13,1,3
1,0,2015,1,2,14,1,3
2,0,2015,1,3,14,1,3
3,0,2015,1,8,7,2,3
4,0,2015,1,13,13,1,3


In [65]:
#filteredCountsDF = countsDataFrame[countsDataFrame['year'].isin([2015, 2016, 2017])]
filteredCountsDF = countsDataFrame
filteredCountsDF.shape

(179586, 7)

In [30]:
from sklearn import linear_model

x_train = filteredCountsDF[['month', 'day', 'hour']].values.reshape(-1, 3)
y_train = filteredCountsDF['counts']

ols = linear_model.LinearRegression()
model = ols.fit(x_train, y_train)
print(model.coef_)

[ 0.00302618 -0.00137503  0.03293429]


In [67]:
X = filteredCountsDF.drop(['counts', 'year', 'day'], axis=1)
y = filteredCountsDF['counts']
X.head(5)

,division,month,hour,dayofweek
0,0,1,13,3
1,0,1,14,3
2,0,1,14,3
3,0,1,7,3
4,0,1,13,3


In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [69]:
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor()  
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [70]:
y_pred = regressor.predict(X_test) 
resultDF=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
resultDF.head(10)

,Actual,Predicted
125877,4,3.333333
55814,4,1.880000
160023,1,2.642857
142356,1,3.666667
136171,5,3.312500
87143,3,3.000000
74493,1,2.500000
23562,1,1.461538
113200,2,3.285714
33911,1,1.571429


In [71]:
from sklearn import metrics 
import numpy as np

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.96862209109866
Mean Squared Error: 1.7217924306962942
Root Mean Squared Error: 1.312170884715971
